# GSL : on the pointers side

Many problems related to pointers come from their multiple uses, in particular the fact that they can indifferently designate memory areas of the stack or the heap, single objects or arrays, dynamic memory areas (heap) managed elsewhere, or to release at the end of the instructions current block. The GSL provides small tools and recommendations to better differentiate all uses.

## Pointers and raw references : `T*` et `T&`

A "raw", or "nude" pointer, (e.g. `T*`) is supposed to have its most ordinary meaning: it points to an object, but does not "own" it. Implied: the portion of code that uses this pointer is not supposed to do a `delete` at the end of use.

If the pointer is not meant to change value, and is never meant to be null, a reference will be preferable whenever possible.

In addition, it is forbidden to use a raw pointer to designate an array. Now, many other solutions exist.

## Non-null Pointers : `gsl::not_null<T>`

When a pointer is going to be dereferenced (via `*` or `->`), a common practice is to first check that it is not null, as a precaution, which obscures and slows down the code. Whenever possible, replace this pointer with a reference.

Otherwise, use `gsl::not_null<T>`, which ensures that you never give the pointer a null value. `T` is usually a pointer type, but in fact` T` can be any type for which `== nullptr` makes sense.

If one tries to assign a null value, the kind of error may vary:
* a compilation error: if `nullptr` is directly assigned to the instance of `gsl::not_null` (`d4` in the example below),
* a runtime error: if `nullptr` goes through an intermediate value (`d3` in the example below).

In [ ]:
%%file tmp.gsl-pointers.cpp

#include <iostream>
#include <gsl/gsl>

class Demo {
  public:
    Demo() { std::cout<<"Constructor"<<std::endl ; }
    ~Demo() { std::cout<<"Destructor"<<std::endl ; }
} ;

int main() {
    gsl::not_null<Demo *> d1 { new Demo() } ;
    delete d1 ;
    // ...
    Demo * d2 = nullptr ;
    // ...
    gsl::not_null<Demo*> d3 { d2 } ;
    // ...
    gsl::not_null<Demo*> d4 { nullptr } ;
}

In [ ]:
!g++ -std=c++17 -I./ tmp.gsl-pointers.cpp -o tmp.gsl-pointers.exe

In [ ]:
!./tmp.gsl-pointers.exe

## Owning pointers : `gsl::owner<T>`

The `owner<T>` type is used to mark that a pointer owns the pointed object. `T` is assumed to be a pointer type, such as `int *`.

The `owner<T>` type does nothing in itself: the call to `delete` must be always done, but this emphasizes the intention, and can allow static checking tools to detect a forgotten delete. Of course, unless a specific reason, it is better to use the standard library smart pointers (see below).

In [ ]:
%%file tmp.gsl-pointers.cpp

#include <iostream>
#include <gsl/gsl>

class Demo {
  public:
    Demo() { std::cout<<"Constructor"<<std::endl ; }
    ~Demo() { std::cout<<"Destructor"<<std::endl ; }
} ;

int main() {
    gsl::owner<Demo*> d = new Demo() ;
    delete d ;
}

In [ ]:
!g++ -std=c++17 -I./ tmp.gsl-pointers.cpp -o tmp.gsl-pointers.exe

In [ ]:
!./tmp.gsl-pointers.exe

## Improved pointers : `std::unique_ptr<T>`, `std::shared_ptr<T>`

In the case of owning pointers, the GSL encourages the use of pointers from the standard library : `std::unique_ptr<T>` et `std::shared_ptr<T>`.

`std::unique_ptr<T>` is optimized and have no cost. It is based on the semantics of displacement and is not copiable. It is an improved version of the old `auto_ptr`.

`std::shared_ptr<T>` is easier to use, because it can be copied at will. As a counterpart, its implementation is complex and its performance bad. Using it as the default pointer is almost like using a "garbage collector". Prefer `std::unique_ptr<T>` whenever possible.

Also, when creating a raw pointerwith `new`, try to transfer it directly to one of these standard enhanced pointers. Only use `owner<T>` when `std::unique_ptr<T>` and `std::shared__ptr<T>` cannot be.

## To remember

Non-proprietary pointers/references, in order of preference :
* `T&` : non-owner, cannot be null (always attached to an element).
* `gsl::not_null<T>`: non-owner, T is a pointer and can't be null.
* `T*` : non-owner, can be null, supposed to point to a unique element.

Proprietary pointers/references, in order of preference :
* `std::unique_ptr<T>` : single owner, not copyable but movable, automates delete, efficient.
* `std::shared_ptr<T>` : shareable owner, automates delete , simple but less efficient.
* `gsl::owner<T>` : owner, T is a pointer, can be null, supposed to point to a dynamically allocated element (on the heap).`

# Exercise

In the code below, add the lacking classes :
* `my_owner` must simply emulate `gsl::owner` (trivial) ;
* `my_not_null` must simply emulate `gsl::not_null` (easy) ;
* make your classes only accept pointers as parameters (complex).

<!-- Solution

#include <cassert>

template <typename T, std::enable_if_t<std::is_pointer_v<T>, int> = 0>
using my_owner = T ;

template <typename T, std::enable_if_t<std::is_pointer_v<T>, int> = 0>
class my_not_null
 {
  public :
    my_not_null( T p ) : m_p(p) { assert(p) ; } 
    my_not_null & operator=( T p ) { assert(p) ; p_ = p ; return *this ; } 
    T operator->() { return p_ ; } 
  private :
    T m_p ;
 } ;

-->

In [ ]:
%%file tmp.gsl-pointers.cpp

#include <iostream>
#include <type_traits>

class Demo
 {
  public:
    Demo() { std::cout<<"Constructor"<<std::endl ; }
    void Display() { std::cout<<"Display"<<std::endl ; }
    ~Demo() { std::cout<<"Destructor"<<std::endl ; }
 } ;


//... my_owner ...

//... my_not_null ...

int main()
 {
  //my_owner<Demo> d1 ;              // COMPILATION ERROR, because Demo is not a pointer
  //my_not_null<Demo*> p1 ;          // COMPILATION ERROR, because p1 doesn't have an intial value
  //my_not_null<Demo*> p2(nullptr) ; // COMPILATION ERROR, because p2 cant' be null
  my_owner<Demo *> d2 = new Demo() ;
  my_not_null<Demo *> p3 = d2 ; 
  //p3 = nullptr ;                   // EXECUTION ERROR, because p3 can't be null
  p3->Display() ;
  delete d2 ;
 }

In [ ]:
!g++ -std=c++17 -I./ tmp.gsl-pointers.cpp -o tmp.gsl-pointers.exe

In [ ]:
!./tmp.gsl-pointers.exe

## References
* http://isocpp.github.io/CppCoreGuidelines/CppCoreGuidelines#gsl-guidelines-support-library
* http://modernescpp.com/index.php/c-core-guideline-the-guidelines-support-library
* http://nullptr.nl/2018/08/refurbish-legacy-code/

© *CNRS 2020*  
*Assembled and written in french by David Chamont, translated by Karim Asnaoui, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)